# Tweem.io Model Persistence

Calibrated model objects to be saved to postgres database table  

### Imports

In [1]:
import os
import pickle

In [2]:
import assembly
from assembly import config as asmbl_config
from assembly import db as asmbl_db
from assembly import models as asmbl_models

### Environment Settings

In [3]:
#  CD out to main app directory root, 
#  This is needed by app context to initialize properly
os.chdir('..')

In [4]:
#  Initialize environment
#  Set env to appropriate location
os.environ['ASSEMBLY_ENV'] = 'Development'
app = assembly.Assembly.init(__name__, {'default': []})

### Inputs

In [5]:
#  Directory where calibrated models are saved locally
MODEL_BASE_DIR = '/Users/liangjh/workspace/tweemio-api/data/models'

In [24]:
#  Calibration persistence parameters
#  trumpian, dempres2020, techbiz, entertainment, comedians
group = 'comedians'
twitter_handles = app.config['SIMILARITY_COMPARISONS'][group]['screen_names']

In [9]:
old_version = 1
new_version = 1

### Save to Database

#### Archive old models

In [12]:
#  Archive old / prior versions, to allow for rollback (if needed)
old_models = asmbl_models.TwmSnModel.query().filter(asmbl_models.TwmSnModel.active == True and asmbl_models.TwmSnModel.grp == group)
for model in old_models:
    model.update(active=False)

#### Save new / latest models

In [25]:
#  Create new
#  Create each model 
for screen_name in twitter_handles:

    print(f'Reading / persisting model for screen_name: {screen_name}')
    with open(f'{MODEL_BASE_DIR}/{group}-{screen_name}.pik', 'rb') as model_file:
        mdl_pkl = pickle.load(model_file)

    asmbl_models.TwmSnModel.create(
        model_name=screen_name,
        version=(1 if (new_version is None) else new_version),
        active=True,
        grp=group,
        pckl=mdl_pkl
    )

Reading / persisting model for screen_name: billmaher
Reading / persisting model for screen_name: conanobrien
Reading / persisting model for screen_name: jimmyfallon
Reading / persisting model for screen_name: jimmykimmel
Reading / persisting model for screen_name: trevornoah
Reading / persisting model for screen_name: jimgaffigan
Reading / persisting model for screen_name: sarahksilverman
Reading / persisting model for screen_name: stevemartintogo
Reading / persisting model for screen_name: stephenathome
Reading / persisting model for screen_name: badbanana
Reading / persisting model for screen_name: jerryseinfeld
Reading / persisting model for screen_name: sethmeyers
Reading / persisting model for screen_name: theellenshow


In [26]:
#  Save Vectorizer for this group
with open(f'{MODEL_BASE_DIR}/{group}-vectorizer.pik', 'rb') as model_file:
    mdl_pkl = pickle.load(model_file)

asmbl_models.TwmSnModel.create(
        model_name='transformation-countvectorizer',
        version=1, active=True, grp=group, pckl=mdl_pkl)

<TwmSnModel>

In [ ]:
# --- TESTING ---
# --- Retrieve Models ---

#  Load / initialize model etc from database
mdl_row = asmbl_models.TwmSnModel.get(11)
mdl_row.pckl

# Query for a specific type
mdls = asmbl_models.TwmSnModel.query().filter(asmbl_models.TwmSnModel.grp =='default' and asmbl_models.TwmSnModel.active == True)
mdl_map = {mdl.screen_name:mdl.pckl for mdl in mdls}